## RSA - An Exploration Of The Mathematics

<hr />

# Table of Contents

### 1. Introduction  (5 points)

### 2. RSA Code Package 

### 3. Code Demo

### 5. Project Narrative AND results of exchanging codes with others. 
### 6. Testing and comparing FME to mod. 

### 7. Breaking codes without a private key 

### 8. Advanced options 







# Introduction to the RSA Encryption System

The RSA encryption system was named after the three people, Rivest, Shamir, and Adleman, who invisioned it in 1977. Before RSA, one party would need to securely distribute secret keys to each of the parties who wish to encrypt data to send to them.  If the secret key was compromised by the outside party or in transit, the encryption would be compromised entirely.

To solve this problem, researchers sought a system where one party could send encrypted messages to another, and even if the method and key being used to encrypt were discovered, those messages would be impossible to decode. 

RSA was born as a solution and the basics of this encryption scheme are included in this project.  Inside this document you will find a mix of functions and code to encrypt and decrypt, as well as break simple encryptions, and a deep dive into some of the basic mathematics and mechanics behind the scheme.

## RSA Basics

Before RSA, if an encryption key got compromised, the entire scheme would be compromised.  With RSA, however, parties can receive all the encrypted messages they want as long as they have the private key, and they do not need to worry about the people sending messages accidentally compromising their encryption key.

The key to the RSA encryption system lies in the fascinating relationship between composite numbers, their prime number factorizations, and modular mathematics.  To create an encryption scheme like RSA, researchers needed to find an function which was easy to calculate, but difficult to reverse once calculated.  This function would take in messages, transformed into their number represenations, and transform them into new, difficult to decipher numbers, creating the encryption.  They were able to find such a function using modular math and exponentiation.

Further down in this project I will show you more of how RSA works and the mathematics behind it.

![image](RSAEncryptionGraphic.png)

## Basic Tool Set Functions

Below is a quick overview of the basic functions I coded to support the encryption process.  RSA requires us to convert alphanumeric characters into their decimal representation in order to perform the encryption process.  Finding and using the binary expansion of a number becomes extremely important when calculating values to a large exponent modulo another number.

  **convert_text()**  - Converts a string message comprised of alphanumeric characters into an ordered list of their decimal ASCII representations.
  * uses ord() for conversion
<br></br>
* **convert_num()**  - Converts a list of message blocks in decimal representation into their ASCII alphanumeric characters and outputs them as one string.
  * uses chr() for conversion
<br></br>
* **convert_binary_string()** -  Converts a decimal number into its binary expansion by dividing it successively by 2 and storing the remainder.
  * outputs a string of the number's bits


In [53]:
def convert_text(_string):
    """
    Define this function such that it takes in a simple 
    string such as "hello" and outputs the corresponding
    standard list of integers (ascii) for each letter in the word hello.
    For example:
    _string = hello
    integer_list = [104, 101, 108, 108, 111]
    """
    integer_list = [] #intilizing the list so we may append it

    #Converting the string to ascii character by character so each character can be captured in a the list
    for char in _string:

        ascii_code = ord(char)  #Convert character to ascii integer representation using ord() method.  int() does not work.
        integer_list.append(ascii_code) #adding the result to the list

    return integer_list

In [54]:
def convert_num(_list):
    """
    Do the opposite of what you did in the Convert_Text
    function defined above.
    
    Define this function such that it takes in a list of integers
    and outputs the corresponding string (ascii).
    
    For example:
    _list = [104, 101, 108, 108, 111]
    _string = hello
    """
    _string = '' #initializing the string variable so we may add to it

    for i in _list: #iterate through each number in the list
        _string += chr(i) #convert number to ascii character using built in python method and concatenate it to the string
        
    return _string

In [55]:
def convert_binary_string(_int):
    """
    Here, you need to define a function that converts an integer to
    a string of its binary expansion.
    
    For example:
    _int = 345
    bits = 101011001
    
    return bits
    """

    dec_num = int(_int) #cast _int to an integer just in case it's a string

    
    if dec_num >= 0:  #make sure num is greater than zero as we were not given the algorithm for negative numbers
        k = 0 #variable to hold bit digits
        bits = "" #initialize bit string variable
    
        #This is the decimal to binary algorithm
        while dec_num > 0 :
            k = dec_num%2 #we take the decimal number modulous 2, the base of binary to find bit digit k
            
            #we find our bits in reverse order from least significant bit to most significant bit
            bits = str(k) + bits #convert to string and add the bit found to the front of the string to reverse order
            
            #advance the algorithm by finding how many times 2 goes into the current decimal number
            dec_num = dec_num//2

        return bits


## Fast Modular Exponeniation, The Euclidean And Extended Euclidean Algorithm

* **FME()** - The fast modular exponentiation algorithm which computes $b^n \mod m$ more efficiently than python alone.  Returns the result as a single integer.
* **Regular_Euclidean_Algorithm()** - Returns the greatest common divisor(gcd) of integers a and b.  
* **Euclidean_Alg(a, b)** - Uses the Extended Euclidean Algorithm for integers a and b to return a tuple of (gcd, s, t), where s and t are the Bezout coefficients and s is our decryption private key d. 

In [56]:
def FME(b, n, m):
    #Fast Modular Exponentiation Algorithm
    #Important notes about it:
    #A MUCH faster way to compute b^n mod m than multiplying all those numbers out
    
    bin_expansion = convert_binary_string(n) #We will get the binary version of the number by calling our conversion function

    rev_expansion = bin_expansion[::-1] #create a reversed string of the binary translation so we may iterate from least significant digit to most significant
    
    x = 1 #
    power = b%m

    for bit in rev_expansion:
        if bit == '1':
            x = (x*power)%m

        power = (power*power)%m

    return x

In [57]:
def Regular_Euclidean_Alg(a, b):
    #I am coding the regular EA so I can practice for the Extended EA

    #First I find which number is larger and assign that to the variable x, this is really just to order the numbers properly for finding the gcd
    if b > a:
        x = b
        y = a
    else: #If they are the same or a is more than b, we assign x to a 
        x = a
        y = b

    #This loops looks for the gcd
    while y != 0: #Our terminating condition is y = 0, or when there is no remainder, as y is assigned to the remainder in the last statement of the loop
        k = x%y # k equals x modulo y, it is the remainder
        x = y #we assign x to the smaller number
        y = k #assign modulo operator to the remainder for next round

    return x #the gcd is the first number in the algorith we were able to divide by and get no remainder

In [58]:
def Euclidean_Alg(a, b):
    """
    1. Calculate the Greatest Common Divisor of a and b.
    
    2. Unless b==0, the result will have the same sign as b (so that when b is divided by it, the result comes out positive).
    The function must return a single integer 'x' which is
    the gcd of a and b.
    """
    #For simplicity's sake, I am assigning a and b to m and n.  
    #It made it easier to recreate the algorithm from Sriram's video and it makes things cleaner since we will alter m and n.
    
    #First I find which number is larger and reorder them in case they were not entered in with the larger one first
    if b > a:
        m = b
        n = a
    else: #If they are the same or a is more than b, we assign m to a and n to b.
        m = a
        n = b


    #To find the Bezout's Coefficients for the gcd of a and b, we will go through many linear combinations of s and t
    s1, t1 = 0, 1   #s1 = 1 and t1 = 0 for the first linear combination
    s2, t2 = 1, 0   #s2 = 0 and t2 = 1 for our second linear combination


    #GCD and Bezout Coefficients finding loop
    while n > 0 :  #We will iterate until n is reduced through the algorithm to 0, in other words, there is no remainder
        k = m%n #remainder, k = m - q*n
        q = m//n #quotient of m = q*n + k

        m = n #We assign m to n to advance the algorithm
        n = k #n is set to the remainder

        #Because we need to reassign the values of s1, t1 and s2, t2, the bezout coefficients,
        # using their values from the previous loop, we need a dummy variable to hold their current values
        s_pr1 = s1 
        t_pr1 = t1

        #Assigning the bezout coefficients to s2 and t2 to advance the algorithm
        s1, t1 = s2, t2

        #Now we get to calculate some new bezout coefficients
        s2 = s_pr1 - q*s2 #we must use the value of s1 before it's last assignment, that is s_pr1
        t2 = t_pr1 - q*t2 #same thing here for t1 as t_pr1

    return m, s1, t1

### 2.3
#### Second tool set

Here we will implement the meat of the RSA cryptosystem. The functions below will generate the public and private key pairs which will then be used to create a ciphertext using the public key and then decode the same using the private key.



## Finding Public and Private Keys

* **Find_Public_Key_e()** - Takes in two prime numbers p and q, and uses the value of (p-1)(q-1) to find e, a relatively prime number to (p-1)(q-1).  Uses the Euclidean Algorithm to test for relatively prime numbers and it makes sure e is not the same as p and q.  The function then returns the public key (n, e).  
* **Find_Private_Key_d()** - Uses the Extended Euclidean Algorithm on e and (p-1)(q-1) to find the private key (n, d).  

In [59]:
def Find_Public_Key_e(p, q):
    """
    Implement this function such that
    it takes 2 primes p and q.
    
    Use the gcd function that you have 
    defined before.
    
    The function should return 2 elements as follows:
    public key: n
    public key: e
    
    HINT: this function will run a loop to find e such 
    that e is relatively prime to (p - 1) (q - 1) 
    and not equal to p or q.
    """

    n = p*q #First part of the public key are two numbers multiplied together
    p_1q_1 = (p-1)*(q-1)

    #the program can run through numbers up to 1000
    for num in range(100, 1000):
    
        #Function returns tuple so we want to grab only the first digit
        #That is the gcd
        gcd_s_t = Euclidean_Alg(num, p_1q_1)
        gcd = gcd_s_t[0]

        if num == p or num == q:
            pass #we don't want our e to be equal to p or q or it would compromise our encryption scheme
            #So we tell it to pass and move on to the next number
        elif gcd == 1:  #If gcd is one then the two numbers are relatively prime
            e = num #we found an e
            break

    return n, e

In [60]:
def Find_Private_Key_d(e, p, q):
    """
    Implement this method
    to find the decryption exponent d such that
    d is the modular inverse of e. 
    
    This will use the Extended Euclidean Algorithm
    
    This function should return the following:
    d: the decryption component.
    """
    #This function finds d, the Bezout coefficient which is big math talk
    # for the secret part of the secret key which inverts our encryption function
    
    #e is relatively prime to (p-1)(q-1) so we calculate it
    p_1q_1 = (p-1)*(q-1) 

    #the Extended Euclidean Algorithm can find d
    gcd_s_t = Euclidean_Alg(e, p_1q_1)

    d = gcd_s_t[1] #Euclidean_Alg returns a tuple of (gcd, d, t) 

    return d

In [61]:
Find_Private_Key_d(17, 23, 31)

233

In [62]:
def Encode(n, e, message):
    """
    Here, the message will be a string of characters.
    Use the function Convert_Text from 
    the basic tool set and get a list of numbers.
    
    Encode each of those numbers using n and e and
    return the encoded cipher_text.
    """
    cipher_text = [] #the ciphered text blocks will be stored here

    num_message = convert_text(message) #Convert the letters to numerical version

    #We must be careful that the numerical value of the message is less than n
    #This helps us encode it without mixing it up with other congruences
    #we need this when we decode it to give the exact same message numerical value 
    #as when we first created it.  This relates to the mechanics of modular math and congruences
    #This loop checks to make sure each block in the message is less than n
    for block in num_message:
        if block >= n: #If it's bigger or equal to n, we create an error message
            print("Error. Messages must be numerically less than the modulo operator, n.")
            return none #exit the function immediately since M is invalid

    #We will iterate through the message units in the message list we fed in
    #This loop calculates the new cipher message for each message block
    for block in num_message:
        cipher = FME(block, e, n) #(message^e mod n) calculates the encrypted version

        cipher_text.append(cipher) #adding to the list of all the cipher blocks
    
    return cipher_text #return the finished list of cipher blocks

In [63]:
print(Encode(5251, 3, 'What song best describes the feeling you get when your RSA code is working?'))

[2128, 1150, 4250, 1349, 1262, 3336, 2371, 2497, 519, 1262, 1263, 1105, 3336, 1349, 1262, 2310, 1105, 3336, 4115, 762, 2405, 1263, 1105, 3336, 1262, 1349, 1150, 1105, 1262, 506, 1105, 1105, 4723, 2405, 2497, 519, 1262, 1974, 2371, 58, 1262, 519, 1105, 1349, 1262, 4839, 1150, 1105, 2497, 1262, 1974, 2371, 58, 762, 1262, 13, 4679, 1573, 1262, 4115, 2371, 2310, 1105, 1262, 2405, 3336, 1262, 4839, 2371, 762, 1560, 2405, 2497, 519, 3250]


In [64]:
def Decode(n, d, cipher_text):
    '''
    Here, the cipher_text will be a list of integers.
    First, you will decrypt each of those integers using 
    n and d.
    Later, you will need to use the function Convert_Num, that converts the integers to a string, 
    from the basic toolset to recover the original message as a string. 
    
    '''
    message = '' #empty string to store final message
    message_blocks = [] #empty list to store message blocks as they are decoded
   
    #This loop 
    for unit in cipher_text:
        num_message = FME(unit, d, n) # message = (cipher**d)modulo n
        message_blocks.append(num_message)
        
    message_blocks = convert_num(message_blocks)

    message = "".join(message_blocks)


    return message

In [65]:
#Decoding a test message
Decode(5251, 3403, [427, 1105, 4723, 4723, 2371, 1262, 2947, 4250, 762, 1349, 1150])

'Hello Earth'

In [66]:
#Professor Stade's piazza Message posted with private key to Decode

Decode(5251, 3403, [2128, 1150, 4250, 1349, 1262, 3336, 2371, 2497, 519, 1262, 1263, 1105, 3336, 1349, 1262, 2310, 1105, 3336, 4115, 762, 2405, 1263, 1105, 3336, 1262, 1349, 1150, 1105, 1262, 506, 1105, 1105, 4723, 2405, 2497, 519, 1262, 1974, 2371, 58, 1262, 519, 1105, 1349, 1262, 4839, 1150, 1105, 2497, 1262, 1974, 2371, 58, 762, 1262, 13, 4679, 1573, 1262, 4115, 2371, 2310, 1105, 1262, 2405, 3336, 1262, 4839, 2371, 762, 1560, 2405, 2497, 519, 3250])

'What song best describes the feeling you get when your RSA code is working?'

In [67]:
#My response to Professor Stade

print(Encode(5251, 3, "Friday by Rebecca Black"))

[1685, 762, 2405, 2310, 4250, 1974, 1262, 1263, 1974, 1262, 13, 1105, 1263, 1105, 4115, 4115, 4250, 1262, 3942, 4723, 4250, 4115, 1560]


# How does RSA and this code work?

RSA relies on creating a function that is easy to calculate in one direction but hard to reverse.  That function is relatively simple and is as follows

$Cipher = (Message)^e \mod n$

This finds a ciphered version of a message when you use the public key n and e.

$Message = (Cipher)^d \mod n$

The formula above, identical in form but with different variables, decrypts the messages, when d and n are the private key(n, d). 


# Overview of the Package

The Encode() and Decode() functions in this package do all the necessary steps to encrypt and decrypt messages, calculating these formulas efficiently using the FME() function.  These are built upon several other functions however so I will walk you through the process.

The first functions in the package help translate text into integer representations and back into letters, and it also includes a function to find the binary expansion of a decimal number.  This is important for the Fast Modular Exponentiation, FME() function.

Below are code examples of calling the different functions in the Basic Tool Set.

In [68]:
#Converting "He" into its decimal representation

convert_text("He") 

[72, 101]

In [69]:
#Reading in a list of integers to be converted to their coerresponding ASCII characters

convert_num([104, 101, 108, 108, 111])

'hello'

In [70]:
#Converting a decimal number to a string of its binary digits

convert_binary_string(42) 

'101010'

In [71]:
#More tests I conducted

convert_binary_string(345)

'101011001'

In [72]:
#The values in the string output is also called the binary expansion

convert_binary_string(19)

'10011'

## Fast Modular Exponentiaton, Regular and Extended Euclidean Algorithms

Using the Euclidean_Alg() and Regular_Euclidean_Algorithm() is nearly the same, except the Regular function only returns the greatest common divisor and not also the Bezout coefficients.  The extended version of the algorithm, implemented in the Euclidean_Alg() is vital for RSA as one of those Bezout coefficients turns out to be private key, d, which can decrypt ciphered messages.  I added the Regular function, though I only needed the extended version, to contrast the two and also for practice programming both.

In [73]:
#Returns only the greatest common divisor, which is 1 in this case

Regular_Euclidean_Alg(27, 1012)

1

## Using the same numbers 27 and 1012 we can compare the extended, Euclidean_Alg()

In [74]:
#Calculates gcd to check for relative primeness and returns secret key d

Euclidean_Alg(27, 1012) #returns coefficients that help us decrypt.  

(1, 75, -2)

## FME calculates b^n mod m

In [75]:
# Calculates 2128^3403 modulo 5251

FME(2128, 3403, 5251)

87

## Finding Your Keys

The requirements for keys are that you create $n = p*q$ where p and q are prime numbers.  The larger the prime numbers you use, the more secure the system is.

The functions in this part of the package take in prime numbers and create e and d in the private and public keys.

### The two keys are:

**Public key (n, e)**  

**Private key (n, d)**

## Encoding and Decoding

After finding private and public keys, you can call the functions Decode() and Encode() to create and decipher ciphers.

## Putting It Together

Below is one complete example of using the package for encryption and decryption.

In [76]:
#First we find the public key with inputs prime numbers (p, q)

Find_Public_Key_e(373, 101)

(37673, 103)

## Now that you have the Private key and d, you can use that to find the Private key

In [77]:
#Then we find the private key with inputs (d, p, q)

Find_Private_Key_d(103, 373, 101)

2167

## We are ready to encrypt and decrypt messages

In [78]:
#Message to encrypt
quote = "A little bit of math can accomplish what all the guns and barbed wire can't: a little bit of math can keep a secret. ― Edward Snowden"

#Let's encrypt this message and pass it to a variable so we may call that later for decoding
snowden_cipher = Encode(37673, 103, quote)

print(snowden_cipher)

[16469, 26405, 17210, 13396, 17010, 17010, 17210, 1212, 26405, 21688, 13396, 17010, 26405, 16453, 36765, 26405, 22429, 35993, 17010, 32347, 26405, 34736, 35993, 8405, 26405, 35993, 34736, 34736, 16453, 22429, 27288, 17210, 13396, 23550, 32347, 26405, 37142, 32347, 35993, 17010, 26405, 35993, 17210, 17210, 26405, 17010, 32347, 1212, 26405, 28692, 20256, 8405, 23550, 26405, 35993, 8405, 9695, 26405, 21688, 35993, 22801, 21688, 1212, 9695, 26405, 37142, 13396, 22801, 1212, 26405, 34736, 35993, 8405, 25989, 17010, 37350, 26405, 35993, 26405, 17210, 13396, 17010, 17010, 17210, 1212, 26405, 21688, 13396, 17010, 26405, 16453, 36765, 26405, 22429, 35993, 17010, 32347, 26405, 34736, 35993, 8405, 26405, 34354, 1212, 1212, 27288, 26405, 35993, 26405, 23550, 1212, 34736, 22801, 1212, 17010, 9365, 26405, 35697, 26405, 8137, 9695, 37142, 35993, 22801, 9695, 26405, 9924, 8405, 16453, 37142, 9695, 1212, 8405]


In [79]:
#Now we will decode the same message

Decode(37673, 2167, snowden_cipher)

"A little bit of math can accomplish what all the guns and barbed wire can't: a little bit of math can keep a secret. ― Edward Snowden"

## We got the same message we put in! Hooray

If you create a custom main function or a script to read codes as your CUSTOM FEATURE, you may include it here with a large heading CUSTOM FEATURE:


CUSTOM FEATURE

$Message = ((Message)^e)^d \mod p \cdot q = (Cipher)^d \mod p \cdot q$

## Code Exchange Examples

I exchanged codes with others online. Here are some examples.

In [80]:
#Piazza post private key is generated for the following examples

# n = 619*569 = 352211

Find_Private_Key_d(83, 619, 569)

164939

In [81]:
#Question I posted to classmates: 'Do you have kids? I have an awesome 10 year old future game designer.'

#The keys used are:
#Public key (n,e) = (352211, 83)
#Private key d = (164939)

my_message = "Do you have any kids? I have an awesome 10 year old future game designer."

print(Encode(352211, 83, my_message))

[143563, 309414, 269424, 189412, 309414, 78087, 269424, 254605, 208357, 113712, 72326, 269424, 208357, 9644, 189412, 269424, 310398, 229607, 291175, 38316, 273079, 269424, 217050, 269424, 254605, 208357, 113712, 72326, 269424, 208357, 9644, 269424, 208357, 107059, 72326, 38316, 309414, 318718, 72326, 269424, 331951, 7162, 269424, 189412, 72326, 208357, 315931, 269424, 309414, 270665, 291175, 269424, 62641, 78087, 315837, 78087, 315931, 72326, 269424, 331307, 208357, 318718, 72326, 269424, 291175, 72326, 38316, 229607, 331307, 9644, 72326, 315931, 344730]


In [82]:
#Testing the encryption I did and my private key

Decode(352211, 164939, [143563, 309414, 269424, 189412, 309414, 78087, 269424, 254605, 208357, 113712, 72326, 269424, 208357, 9644, 189412, 269424, 310398, 229607, 291175, 38316, 273079, 269424, 217050, 269424, 254605, 208357, 113712, 72326, 269424, 208357, 9644, 269424, 208357, 107059, 72326, 38316, 309414, 318718, 72326, 269424, 331951, 7162, 269424, 189412, 72326, 208357, 315931, 269424, 309414, 270665, 291175, 269424, 62641, 78087, 315837, 78087, 315931, 72326, 269424, 331307, 208357, 318718, 72326, 269424, 291175, 72326, 38316, 229607, 331307, 9644, 72326, 315931, 344730])

'Do you have any kids? I have an awesome 10 year old future game designer.'

In [83]:
#Tory answered my question below with these codes using my public key

torys_answer = [217050, 269424, 254605, 208357, 113712, 72326, 269424, 208357, 269424, 321200, 88539, 189412, 72326, 208357, 315931, 88539, 309414, 270665, 291175, 269424, 291175, 208357, 78087, 331307, 254605, 315837, 72326, 315931, 269424, 107059, 254605, 309414, 269424, 208357, 270665, 315931, 72326, 208357, 291175, 189412, 269424, 310398, 9644, 309414, 107059, 38316, 269424, 254605, 72326, 315931, 269424, 107059, 208357, 189412, 269424, 208357, 315931, 309414, 78087, 9644, 291175, 269424, 315837, 254605, 72326, 269424, 286164, 215128, 48964, 131756]

Decode(352211, 164939, torys_answer)

'I have a 4-year-old daughter who already knows her way around the PS5!'

In [84]:
#Taylor's post
#Public key (n,e): 10807, 3
#Private key (d): 7067

ciphers = [10083, 936, 4885, 4688, 347, 1276, 7895, 347, 10020, 5949, 2177, 985, 347, 2122, 4885, 368, 5949, 985, 1276, 4688, 3636, 347, 6100, 4885, 4688, 3636, 4669, 1739, 1276, 1220, 936, 4688, 347, 7895, 1739, 4885, 8476, 3852, 1486]

Decode(10807, 7067, ciphers)

'What is your favorite late-night snack?'

In [85]:
#I said back to Taylor

message = "Bark thins or dried mango!"

print(Encode(10807, 3, message))

[6514, 4885, 985, 3852, 347, 4688, 936, 1276, 1739, 7895, 347, 5949, 985, 347, 5756, 985, 1276, 3636, 5756, 347, 8996, 4885, 1739, 1220, 5949, 3516]


In [86]:
#Raul posted an encrypted message.  He was answering a question in the main prompt
#that said "what is your favorite movie?"

#Public key (n, e): (47959, 109)

#Private key d: 35749

#Encoded message = [15115, 14887, 35173, 23104, 29742, 8486, 18913, 12922, 16512, 23104, 8486, 18913, 44659, 16512]

Decode(47959, 35749, [15115, 14887, 35173, 23104, 29742, 8486, 18913, 12922, 16512, 23104, 8486, 18913, 44659, 16512])


'E.T phone home'

In [87]:
#I responded by to the prompt, about E.T. saying I had never watched the whole thing

print(Encode(47959, 109, "I have never watched that one from start to finish if you can believe it.  Even though I'm a child of the eighties"))

[14675, 23104, 8486, 45897, 44193, 16512, 23104, 12922, 16512, 44193, 16512, 19804, 23104, 35176, 45897, 36109, 30495, 8486, 16512, 22837, 23104, 36109, 8486, 45897, 36109, 23104, 18913, 12922, 16512, 23104, 24549, 19804, 18913, 44659, 23104, 8806, 36109, 45897, 19804, 36109, 23104, 36109, 18913, 23104, 24549, 31709, 12922, 31709, 8806, 8486, 23104, 31709, 24549, 23104, 35547, 18913, 824, 23104, 30495, 45897, 12922, 23104, 42989, 16512, 29937, 31709, 16512, 44193, 16512, 23104, 31709, 36109, 14887, 23104, 23104, 15115, 44193, 16512, 12922, 23104, 36109, 8486, 18913, 824, 44690, 8486, 23104, 14675, 28686, 44659, 23104, 45897, 23104, 30495, 8486, 31709, 29937, 22837, 23104, 18913, 24549, 23104, 36109, 8486, 16512, 23104, 16512, 31709, 44690, 8486, 36109, 31709, 16512, 8806]


# Computing Modular Exponentiation Efficiently

For the RSA system, calculating modular exponentiation is extremely important.

The invertable function which takes a message and converts it to a cipher, as well as its inverse, the decoding function, directly uses modular exponetiation.

For each message block you must run the equation

$Cipher = (Message)^e \mod n$

However, dividing $(Message)^e$ by $n$ to find the remainder can take a long time because $(Message)^e$ may be very large.
<br></br>

If we were to compute the value of something like

$37^{19}\mod 15$

We could instead, start by calculating each value of $37^i \mod 15$ starting with $i = 1$  till  $i = 19$.
  
In modular multiplication, the distributive property requires us to take the mod each time we calculate the next value in the series of  
$37^1\mod 15$  
$37^2 \mod 15 = ((37 \mod 15)*(37 \mod 15)) \mod 15$  
$37^3 \mod 15 = (37 \mod 15 *((7 \mod 15)*(37 \mod 15)) \mod 15)) \mod 15$
<br></br>
This calculation gets very messy and can take too long as well.  In order to be useful, encryption and decryption needs to be able to compute its values efficiently.
<br></br>
## Fast Modular Exponentiation
For this reason I used Fast Modular Exponentiation for my RSA scheme, which uses a trick to help us find the value of 

$Cipher = (Message)^e \mod n$  

as well as finding the value for the Message formula.
<br></br>
From our earlier example of $37^{19}\mod 15$,  converting the exponent into its binary expansion we find that  

$19 = (10011) base2$  
$19 = 16 + 2 + 1$  

and

$37^{19} = (37^{16})*(37^2)*(37^1)$

therefore
  
$37^{19} \mod 15 = (37^{16} \mod 15 * 37^2 \mod 15 * 37 \mod 15) \mod 15$

This dramatically cuts down the number of calculations our algorithm needs to carry out, thus making Fast Modular Exponentiation a better way to calculate our Ciphers and Messages, especially as larger numbers are used for e and n.</p>
  
<br>
Examples
--------

Python calculates the formula 

> (a**b)%c

by first calculating (a**b) and then dividing that repeatedly by  c until it arives at the remainder.

I have included specific examples running such a formula with first with simple numbers and then with a message.

For each example set, I compare the time it takes with the regular, simple python math method and then I use the FME algorithm I coded.

The first set use a simple 3 digit number for the exponent.  With these three numbers, 1802, 109 and modulo 47959, the regular python calculation is faster.

For all the following examples however we will see something different.

In [88]:
# Testing the speed of our FME algorithm on the same numbers
# 1802 to the 109th power modulo 47959

%timeit FME(1802, 109, 47959) #Implement FME to test speed

5.71 µs ± 877 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### What happens if we make our exponent larger?

When I ran our formula with a 6 digit exponent, the runtime increases to $1.64$ full seconds but the FME function only took $40 \mu$ s.  This was a noticeable amount that had me stop a moment.

In [89]:
#Testing regular python calculation

%%time (1802**983041)%47959

UsageError: Line magic function `%%time` not found.


In [90]:
#Same numbers with FME

%%time FME(1802, 983041, 47959)

UsageError: Line magic function `%%time` not found.


## What happens when we want to use the two methods to cipher an entire message?

Below after converting text to a list of its numerical message blocks, I compared feeding the entire list into the straightforward formula and into the FME function.  Then I used (983941, 47959) as my pseudo key to explore both methods.

The difference in time it took to use the FME algorithm, versus letting python do all the computation was astounding!  Only 1.82 ms for FME versus a full 1 min and 22 seconds for the direct python calculation.

Seeing this it is easy to imagine how as crypto systems gets more secure through the use of more digits, we would need to have a faster way of calculating the cipher and decipher functions.

In [91]:
message = "'A little bit of math can accomplish what all the guns and barbed wire can't: a little bit of math can keep a secret.' ― Edward Snowden"

num_message = convert_text(message) #convert message to decimal representation

In [92]:
#Testing regular python calculation for a group of message blocks

%%time

for message_block in num_message:
    (message_block**983941)%47959 #loop to cipher each message block

UsageError: Line magic function `%%time` not found.


In [93]:
#Testing the calculation time of FME for a group of messages

%%time

for message_block in num_message:
    FME(message_block, 983941, 47959)

UsageError: Line magic function `%%time` not found.


# How and why code breaking works for factoring

If you know what the crypto system used is, you can begin to formulate an attack to break it.  Without knowledge of the encryption method used, breaking codes becomes even more difficult.  Thankfully for this project, I am only examining RSA and it's encryption scheme.

When you create a cipher, a function is used to compute the ciphered numerical values for each message block.   The function for RSA, previously discussed, is

$Cipher = (Message)^e \mod n$

We know that $n = p*q$ based on the crypto system and also by the mathemtatical theroem:

> If n is a composite integer, then $n$ has at least one prime divisor less than or equal to $\sqrt{n}$

To defeat RSA, you need to be able to create the inverse function of the encryption function.  

The inverse function of RSA is the same one used to decipher its encrypted messages.  The value $d$ is the inverse of  $e \mod (p-1)*(q-1)$  and the inverse function to decode is

$Message = (Cipher)^d \mod p \cdot q$

If we can factor n into it's two prime factors $p$ and $q$, then we can find $d$ by calculating $(p-1)(q-1)$ and using the Extended Euclidean Algorithm to find the Bezout Coefficients.

$d\space$ turns out to be the Bezout coefficient of $e$ and $(p-1)(q-1)$.


Factoring large composite numbers is famously believed to be difficult and time consuming. Efficient methods have been sought after but the best methods can only be employed with some knowledge of the crypto system used or with special conditions being satisfied by the number being factored.  The time taken by general purpose algorithms factoring n is entirely dependent on the length of the digit n.

In order to increase research in the area of number theory and the difficulty of factorization, the RSA Laboratories created a challenge in 1991 offering cash prizes to those who successfully break the RSA system.  The first challenge, RSA-100, or RSA using a 100 digit decimal number was broken only a couple weeks after the challenge was released.  The factorization took a few days with the technology at the time.

The most recent challenge to be broken was RSA-200 and this was broken in 2020.  The time and computing power it took however is astronimical, equivalent to 2,700 years of runtime on a single CPU.

As numbers become larger and larger, their difficulty in factorization increases exponentially.

To make factoring harder, the RSA system should choose prime numbers p and q at random and let them be only a few digits different in length.  Since the frequency of primes decreases dramatically as integers become larger and larger, choosing primes that are very large and close to each other in length ensures it will take longer to find either of those special prime factors. Once either p or q is found, finding the other prime is as easy as dividing n by the new prime factor.

Sources:

https://en.wikipedia.org/wiki/RSA_Factoring_Challenge

https://en.wikipedia.org/wiki/RSA_numbers#RSA-250

https://en.wikipedia.org/wiki/Integer_factorization


# Factorization Functions for Code Breaking and Prime Testing

In [94]:
def brute_force_factorize(n):
    #This function takes in an integer tests every number from 1 to n to see if n is divisible by it
    #it returns the complete list of factors including 1 and n
    factors = []

    for i in range(1, (n+1)): #loop to check every number from 1 to n. range goes up to but not including n+1
        
        if n%i == 0: #if there's no remainder, then we found a factor
            factors.append(i)
            
    return factors #return the complete list of factors

In [95]:
brute_force_factorize(98)

[1, 2, 7, 14, 49, 98]

### We don't need to factor every number less than n

A mathematical thereom tells us that every positive integer n greater than 2 has at least one unique prime factor less than or equal to the square root of n.

The next algorithm implements this knowledge and returns a list of factors just up to the square root of n, cutting the loop iterations in half.

In [96]:
def less_brutish_factorize(n):
    #This function takes in an integer tests every number from 1 to the square root of n to see if n is divisible by it
    import math
    
    factors = []
    
    sqrt_n = int(math.sqrt(n)) #calculating square root of n
    
    #loop checks every number from 1 to n
    for i in range(1, (sqrt_n + 1)):
        if n%i == 0: #if there's no remainder, then we found a factor
            factors.append(i)
        
            
    return factors #return the complete list of factors

In [97]:
#testing our less brutish algorithm with a 6 digit number

less_brutish_factorize(98)

[1, 2, 7]

# The Growing Importance of Audio Encryption

An area of great interest to me personally is audio synthesis and how that could be used for disinformation.  Every few months a new, open-source machine learning model comes out which can synthesize human voice with greater and greater accuracy.

The applications for voice synthesis are far reaching, from helping independent authors create audiobooks without needing to shell out cash for a reader or spend hours reading themselves, to making the world more accessible for the blind.  With it, mute people can talk and our phones can talk back to us as well.

It also introduces many problems, primarily the problem of people's voices being stolen and synthesized without their approval.  This new ability to synthesize human speech believably has far reaching poilitcal and social implications.  A famous person or politician could have their voice stolen through unencrypted audio clips sitting on the web on youtube or through calls intercepted.  The unathorized party could then use that data and synthesized voice to impersonate said politician and "leak" very believable but entirely fake audio.

We can use RSA to encrypt audio just as we can use it to encrypt text.  To encrypt audio, I created two functions, encode_audio() and decode_and_play_audio(), to take in an audio file and change it into an array of bytes.  It then saves the file in the current folder which you can download to listen to.

The audio clip I use in my encryption example below is taken from a set of audio clips I synthesized using the Flowtron model from Nvidia.  I tried having the file play in the jupyter notebook but could not get that part to work in Jupyterlab.

In [98]:
def encode_audio(audio_file, n, e):
    #To encrypt our audio we can open the file using a simple python with statement
    #And create a byte stream from it to encrypt byte by byte

    with open(audio_file, 'rb') as f:
        contents = f.read()

    #Convert bytestream into integer array
    audio_array = [i for i in contents]

    encrpyted_audio_array = []

    for block in audio_array:
        if block >= n: #If it's bigger or equal to n, we create an error message
            print("Error. Messages must be numerically less than the modulo operator, n.")
            return None #exit the function immediately since M is invalid

    #We will iterate through the audio units in the clip
    #This loop calculates the new cipher message for each message block
    for block in audio_array:
        encrpyted_audio_block = FME(block, e, n) #(message^e mod n) calculates the encrypted version

        encrpyted_audio_array.append(encrpyted_audio_block) #adding to the list of all the cipher blocks
    return encrpyted_audio_array        

In [99]:
import IPython

def decode_and_save_audio(encrypted_audio_array, decrypted_audio_path, n, d):
    decrypted_audio_blocks = [] #empty list to store audio blocks as they are decoded
   
    #This loop decrypts the audio block by block
    for unit in encrypted_audio_array:
        decrypted_audio_block = FME(unit, d, n) # message = (cipher**d)modulo n
        decrypted_audio_blocks.append(decrypted_audio_block)
        
    audio_byte_array = bytes(decrypted_audio_blocks) #Convert int array into byte array
    
    #saving the new decrypted file
    with open(decrypted_audio_path, 'wb') as df:
        df.write(audio_byte_array)


In [100]:
#Let's encrypt using keys from earlier

#The keys used are:
#Public key (n,e) = (352211, 83)
#Private key d = (164939)

encoded_audio = encode_audio("joinedFile-9.wav" , 352211, 83)

In [ ]:
#Now let's decode and save our audio so we can listen to it

decode_and_save_audio(encoded_audio, "decodedJoinedFile-9.wav", 352211, 164939)

# Extra Exploration of the Math and Concepts of RSA

# Modular Inverses

Why are they important?

In RSA we must find our invertable function and Modular Inverses turn out to be the key ingredient.  I created this proof and discussion of the thereom relating to whether inverses exist for a particular n modulo m.


Def:
Let a, n be relatively prime numbers. The number b is called the modular inverse of a modulo n if:

ab $\equiv$ 1 (mod n)


Not all numbers modulo have inverses.  They exist if and only if the above formula can be satisfied by some b.

Thereom: If $a, n \in N$ and are relatively prime numbers, then an inverse b of a modulo n exists, and that b can be written as
ab $\equiv$ 1 (mod n).

I wanted to show you the proof of this one because I liked it so much:

We first let n and b be relatively prime numbers because that is the necessary condition for the thereom.

Because they are relatively prime, by definition, the gcd(a, n) = 1.

By Bezout's Thereom we know there exists a $s, t \in Z$ such that  $s \cdot a$ + $t \cdot n$ = 1.

Solving for $s \cdot a$ we have

$s \cdot a$ = 1 - $t \cdot n$

Now we will take modulo n of both sides

$(s \cdot a ) mod  n = (1 - t \cdot n) mod n$




Distributing the modulo operation we have

$(s \cdot a) mod n = (1 mod n - t \cdot n\ mod n)mod n$



We notice that  

$- t \cdot n\ mod n = 0$

since n divides n evenly.


Therefore,

$(s \cdot a) mod n = (1 mod n)mod n$

And by the identity elements we know

$(1 mod n) mod n = (1)mod n$

Therefore, 

$(s \cdot a) \equiv 1(mod n)$

And we see that   $a(modn)$ has an inverse by definition.

That inverse is the Bezout coefficient which is an amazing find indeed.

The Bezout coefficient gives the modular inverse.

I loved how these two concepts connected at the end of the proof.  One of my favorites so far.